<a href="https://colab.research.google.com/github/alecseiterr/safe_city/blob/main/Taranov_Anatoly/Test_file/HumanFallDetect%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Упрощённый ObjectDetect.
* Вырез фрагмента видео
* Проверка весов обученной сети детекцией на целевом видеофрагменте
* Сохранение результата

In [ ]:
# Открываем доступ к гугл диску
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# ЗАПУСКАТЬ ПО НЕОБХОДИМОСТИ
#import os
#
## Указываем путь к папке, которую нужно создать
#output_directory = '/content/drive/MyDrive/ДИПЛОМ_ТГУ/Mytischi/img_dat_1'
#
## Проверяем, существует ли папка
#if not os.path.exists(output_directory):
#    # Создаем папку, если она не существует
#    os.makedirs(output_directory)
#    print("Папка успешно создана")
#else:
#    print("Папка уже существует")


# ВЫРЕЗАЕМ ФРАГМЕНТ ВИДЕО

In [ ]:
import cv2
import numpy as np
import os
from google.colab import drive

# Подключаем Google Drive
drive.mount('/content/drive')

# Открываем видеофайл
video_path = '/content/drive/MyDrive/ДИПЛОМ_ТГУ/Mytischi/Город Мытищи с высоты птичьего полёта.mp4'
cap = cv2.VideoCapture(video_path)

# Параметры временных промежутков для вырезания
start_time_1 = 118  # начальное время фрагмента (в секундах)
end_time_1 = 128    # конечное время фрагмента (в секундах)

# Создаем директорию для сохранения кадров
output_directory = '/content/drive/MyDrive/ДИПЛОМ_ТГУ/Mytischi/img_dat_4'
os.makedirs(output_directory, exist_ok=True)

# Устанавливаем текущую позицию видео на начальное время первого фрагмента
cap.set(cv2.CAP_PROP_POS_MSEC, start_time_1 * 1000)

# Читаем и сохраняем кадры фрагмента
frame_count = 0
frames = []
while cap.isOpened() and cap.get(cv2.CAP_PROP_POS_MSEC) <= end_time_1 * 1000:
    ret, frame = cap.read()
    if ret:
        frame_count += 1
        frames.append(frame)
        # Сохраняем кадр в указанную директорию
        cv2.imwrite(os.path.join(output_directory, f"frame_{frame_count}.jpg"), frame)
    else:
        break
# Создаем видеофайл из кадров
output_video_path = '/content/drive/MyDrive/ДИПЛОМ_ТГУ/Mytischi/вырезанный_фрагмен_4.mp4'  # Путь сохранения вырезанного видео фрагмента на гугл диске
height, width, _ = frames[0].shape
fourcc = cv2.VideoWriter_fourcc(*'mp4v')                                  # Кодек может дурковать запускать второй раз
#fourcc = cv2.VideoWriter_fourcc(*'H264')
out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (width, height))

for frame in frames:
    out.write(frame)

# Освобождаем ресурсы
cap.release()
out.release()
cv2.destroyAllWindows()

# Выводим для проверки и просмотра
#import IPython.display as display
#display.Video(output_video_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Вывод на экран
#import IPython.display as display
#display.Video(output_video_path)


# Упрощённый ObjectDetect

Загружаем модель

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.226 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.0/107.7 GB disk)


In [ ]:
import cv2
from ultralytics import YOLO

# Загрузка модели YOLOv8
model = YOLO("yolov8m.pt", "/content/drive/MyDrive/MyDIPLOM/Weights/best.pt")  # Модель, "/путь к вашей папки с обученными весами /Weights/best.pt  (или путь к вашей обученной модели)

# Открытие видеофайла
video_path = "/content/drive/MyDrive/MyDIPLOM/Mytischi/вырезанный_фрагмент.mp4"  # Путь к вашему вырезанному видео фрагменту на вашем гугл диске
cap = cv2.VideoCapture(video_path)

# Определение параметров для сохранения видео
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = '/content/drive/MyDrive/MyDIPLOM/Mytischi/track_result1.mp4'      # Путь к видео фрагменту c результатом работы Object Detect в папке "track_result1.mp4"на вашем гугл диске
out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

# Перебор кадров видео
while cap.isOpened():
    success, frame = cap.read()
    if success:
        # Выполнение трекинга с помощью YOLOv8 на кадре
        results = model.track(frame, persist=True)

        # Визуализация результатов на кадре
        annotated_frame = results[0].plot()

        # Сохранение аннотированного кадра
        out.write(annotated_frame)

        # Выход из цикла при нажатии клавиши 'q'
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Выход из цикла при достижении конца видео
        break

# Закрытие объекта захвата видео и записи
cap.release()
out.release()
cv2.destroyAllWindows()

# Вывод пути к сохраненному видео
print(f"Видео сохранено по пути: {output_path}")


# Скачиваем файл c результатом работы Object Detect из папки "track_result1.mp4"